# Import Statements and Loading Data

In [1]:
#Import statements
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import gc

max_length = 200

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2024-07-26 08:00:14.831300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 08:00:14.845025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 08:00:14.845046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 08:00:14.854814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 08:00:15.539767: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-26 08:00:17.602371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 08:00:17.605829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 08:00:17.608704: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# BERT Base Model


In [2]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)


2024-07-26 08:00:17.809326: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 08:00:17.812467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 08:00:17.815108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
train_df = pd.read_csv('data/medal_smaller_train.csv')
val_df = pd.read_csv('data/medal_smaller_validation.csv')
test_df = pd.read_csv('data/medal_smaller_validation.csv')

samples = train_df.label.value_counts().sort_values(ascending=False)[:6500].index #6500
samples


Index(['physical therapy', 'respiratory insufficiency', 'footpad',
       'left anterior descending coronary', 'myoinositol', 't cells',
       'dermal papilla', 'peak plasma concentrations', 'kinasedead', 'cbl',
       ...
       'pressurized liquid extraction', 'leschnyhan', 'raynauds disease',
       'biliopancreatic diversion', 'enteroglucagon', 'gastric fistula',
       'insideout vesicles', 'transcranial doppler ultrasound',
       'prothrombin times', 'highfrequency oscillatory ventilation'],
      dtype='object', name='label', length=6500)

In [4]:
train_df = train_df.loc[train_df['label'].isin(samples)]
val_df = val_df.loc[val_df['label'].isin(samples)]

label_names = (list(train_df.label) +
               list(val_df.label))
label_names = sorted(list(set(label_names)))
offset = bert_tokenizer.vocab_size
label_dict = {value: index + offset for index, value in enumerate(label_names)}
reverse_label_dict = {value: key for key, value in label_dict.items()}
print(f"Size: {len(label_names)}", label_names)
print(f"Dict Item 1: {list(label_dict.items())[0]}")
print(f"Reverse Dict Item 1: {list(reverse_label_dict.items())[0]}")

print("Train dataset length:", len(train_df))
print("Validation dataset length:", len(val_df))

with open('dictionary.json', 'w') as json_file:
    json.dump(label_dict, json_file)

with open('reverse_dictionary.json', 'w') as json_file:
    json.dump(reverse_label_dict, json_file)

with open('label_names.json', 'w') as json_file:
    json.dump(label_names, json_file)
    

Size: 6500 ['abasic', 'abdominal', 'abdominal aortic aneurysm repair', 'abdominal compartment syndrome', 'abdominal fat', 'abdominal hysterectomy', 'abductor pollicis brevis', 'ablebodied', 'abnormalities', 'aboveknee', 'abscess', 'abscisic acid', 'absence epilepsy', 'absence seizures', 'absolute deviation', 'absolute error', 'absorption rate', 'academic medical center', 'acanthamoeba keratitis', 'acanthosis nigricans', 'accelerated phase', 'accessible surface area', 'accessory pathways', 'acebutolol', 'acellular dermal matrix', 'acetone', 'acetoxymethyl ester', 'acetylcholinesterase activity', 'acetyltransferase', 'achilles tendon', 'aciclovir', 'acid mine drainage', 'acid resistance', 'acid synthesis', 'acidic protein', 'acidity', 'acoustic emission', 'acoustic reflex', 'acoustic rhinometry', 'acquired immune deficiency', 'acquired immune deficiency syndrome', 'acquired immunodeficiency', 'acridine orange', 'acrolein', 'acromioclavicular', 'acrosomereacted', 'actin filaments', 'actin

In [5]:
def tokenize(dataset, tokenizer=bert_tokenizer, max_len=max_length, label_dict=label_dict):
    input_ids = []
    token_type_ids = []
    start_positions = []
    end_positions = []
    attention_masks = []
    label_ids = []

    for text, loc, abbreviation, label in zip(dataset['text'], dataset['location'], dataset['abbreviation'], dataset['label']):
        pre_tokens = tokenizer.tokenize(' '.join(text.split()[:loc]))
        adjusted_loc_start = len(pre_tokens) + 1
        adjusted_loc_end = adjusted_loc_start + len(tokenizer.tokenize(abbreviation))
        encoded_input = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        
        if adjusted_loc_end < max_length:
            input_ids.append(encoded_input['input_ids'])
            token_type_ids.append(encoded_input['token_type_ids'])
            start_positions.append(adjusted_loc_start)
            end_positions.append(adjusted_loc_end)
            attention_masks.append(encoded_input['attention_mask'])
            label_ids.append(label_dict[label])

    input_ids = np.array(input_ids, dtype=np.int32).squeeze()
    token_type_ids = np.array(token_type_ids, dtype=np.int32).squeeze()
    attention_masks = np.array(attention_masks, dtype=np.int32).squeeze()
    start_positions = np.array(start_positions, dtype=np.int32).squeeze()
    end_positions = np.array(end_positions, dtype=np.int32).squeeze()
    label_ids = np.array(label_ids, dtype=np.int32).squeeze()

    print("First text:\n", dataset['text'].iloc[0])
    print("First location:", dataset['location'].iloc[0])
    print("First acronym:", dataset['text'].iloc[0].split()[dataset['location'].iloc[0]])
    print("First expansion:", dataset['label'].iloc[0])
    print("First text decoded:\n", tokenizer.decode(input_ids[0]))
    print("Confirm adjusted location accuracy: \n",
          tokenizer.decode(input_ids[0][start_positions[0]:end_positions[0]]))
    print("Confirm label:", reverse_label_dict[label_ids[0]])

    return input_ids, token_type_ids, attention_masks, start_positions, end_positions, label_ids

In [6]:
print("Train-------------------------------------------------------------")
train_input_ids, train_token_type_ids, train_attention_masks, train_start_positions, train_end_positions, train_labels = tokenize(train_df)
print("Val---------------------------------------------------------------")
val_input_ids, val_token_type_ids, val_attention_masks, val_start_positions, val_end_positions, val_labels = tokenize(val_df)

np.save('inputs/train_input_ids.npy', train_input_ids)
np.save('inputs/train_token_type_ids.npy', train_token_type_ids)
np.save('inputs/train_attention_masks.npy', train_attention_masks)
np.save('inputs/train_start_positions.npy', train_start_positions)
np.save('inputs/train_end_positions.npy', train_end_positions)
np.save('inputs/train_labels.npy', train_labels)
np.save('inputs/val_input_ids.npy', val_input_ids)
np.save('inputs/val_token_type_ids.npy', val_token_type_ids)
np.save('inputs/val_attention_masks.npy', val_attention_masks)
np.save('inputs/val_start_positions.npy', val_start_positions)
np.save('inputs/val_end_positions.npy', val_end_positions)
np.save('inputs/val_labels.npy', val_labels)

del train_df
del val_df
gc.collect()

Train-------------------------------------------------------------
First text:
 the clinical features of our cases demonstrated some of the already known characteristics of the variable spectrum of hiv infection da are the most important risk category in italy of the arc cases evolved into aids during a month followup on average the most frequent oi in our aids cases were pcp c albicans esophagitis and chronic mucocutaneous ulcers an high percentage of neurologic involvement from hiv was observed and HM were encountered in aids ks and undifferentiated b lymphoma as well as in arc HD statistically significant worsening of the immunologic situation is evident as the disease progresses from las to aids G1 b lymphocytes represent most of the cells of the germinal center during the hyperplastic stage of lymphadenopathy reversal of the tt ratio appears early during the initial stage of lymphadenopathy and is due to a decrease of cd and a relative increase of cd also destruction of the follic

0

In [7]:
@tf.keras.saving.register_keras_serializable()
class ExtractAbbreviationHiddenStates(tf.keras.layers.Layer):
    """
    Custom layer that extracts abbreviation embeddings from BERT
    hidden layer state and position         padded_slices = padded_array[:, :inputs
    """
    def call(self, inputs):
        last_hidden_state, start_abbrev_token_positions, end_abbrev_token_positions = inputs

        batch_size = tf.shape(last_hidden_state)[0]
        max_length = tf.shape(last_hidden_state)[1]

        mask = tf.range(max_length)
        mask = tf.tile(mask[tf.newaxis, :], [batch_size, 1])
        mask = tf.logical_and(mask >= start_abbrev_token_positions, mask < end_abbrev_token_positions)
        span_hidden_state = tf.where(tf.expand_dims(mask, -1), last_hidden_state, tf.zeros_like(last_hidden_state))

        return span_hidden_state

def create_bert_multiclass_model(checkpoint = model_checkpoint,
                                 num_classes = len(label_names),
                                 learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the pooled abbreviation
    token embeddings for classification purposes.
    """
    tf.keras.backend.clear_session()

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')
    start_abbrev_token_positions = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='start_abbreviation_token_positions_layer')
    end_abbrev_token_positions = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='end_abbreviation_token_positions_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_model = TFBertModel.from_pretrained(checkpoint, from_pt=True)
    bert_model.trainable = True

    bert_out = bert_model(bert_inputs)

    last_hidden_state = bert_out.last_hidden_state

    span_hidden_states = ExtractAbbreviationHiddenStates()([last_hidden_state, start_abbrev_token_positions, end_abbrev_token_positions])

    pooled_output = tf.reduce_mean(span_hidden_states, axis=1)

    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(pooled_output)

    classification_model = tf.keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask, start_abbrev_token_positions, end_abbrev_token_positions],
        outputs=[classification],
    )

    def custom_loss(y_true, y_pred, offset=offset):
        y_true = y_true - offset
        return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)
    
    def custom_accuracy(y_true, y_pred):
        offset = bert_tokenizer.vocab_size
        y_true_adjusted = y_true - offset
        return tf.keras.metrics.sparse_categorical_accuracy(y_true_adjusted, y_pred)
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=custom_loss,
                                 metrics=[custom_accuracy])

    return classification_model


In [8]:
model = create_bert_multiclass_model()
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                      

In [9]:
train_inputs = [
    train_input_ids,
    train_token_type_ids,
    train_attention_masks,
    train_start_positions,
    train_end_positions,
]
valid_inputs = [
    val_input_ids,
    val_token_type_ids,
    val_attention_masks,
    val_start_positions,
    val_end_positions,
]
history = model.fit(
    train_inputs,
    np.array(train_labels),
    validation_data=(valid_inputs, np.array(val_labels)),
    batch_size=16,
    shuffle=True,
    verbose=1,
    epochs=5,
)

model.save('models/20240725_base_bert_ft')
model.save_weights('models/20240725_base_bert_ft_weights.hdf5')

Epoch 1/5


I0000 00:00:1722000090.515953 2075669 service.cc:145] XLA service 0x7c29d5976dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722000090.515997 2075669 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-26 09:21:30.524722: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 09:21:30.553086: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1722000090.593535 2075669 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


86405/86405 [==============================] - 16165s 187ms/step - loss: 6.8313 - custom_accuracy: 0.2909 - val_loss: 3.7683 - val_custom_accuracy: 0.3404
Epoch 2/5
86405/86405 [==============================] - 16313s 189ms/step - loss: 2.3104 - custom_accuracy: 0.3928 - val_loss: 1.5147 - val_custom_accuracy: 0.5018
Epoch 3/5
86405/86405 [==============================] - 16484s 191ms/step - loss: 1.2549 - custom_accuracy: 0.5955 - val_loss: 0.9794 - val_custom_accuracy: 0.6856
Epoch 4/5
86405/86405 [==============================] - 16015s 185ms/step - loss: 0.8769 - custom_accuracy: 0.7245 - val_loss: 0.6976 - val_custom_accuracy: 0.7629
Epoch 5/5
86405/86405 [==============================] - 16027s 185ms/step - loss: 0.6893 - custom_accuracy: 0.7745 - val_loss: 0.6073 - val_custom_accuracy: 0.7864
INFO:tensorflow:Assets written to: models/20240725_base_bert_ft/assets


INFO:tensorflow:Assets written to: models/20240725_base_bert_ft/assets
